## Optimization exercise

**Goal: Train the 2nd order polynomial predictor using gradient descent. Optimize the stepsizes and compare against scikit-learn's implementation.**

In [4]:
# IMPORT PACKAGES
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures

In [3]:
data_path = ''
nb = pd.read_csv(data_path+'nba_games_2013_2015.csv', delimiter=';')
x = nb[['AST','REB','STL']]
y = nb['PTS']

#### Task I
Download the data from [here](https://drive.google.com/file/d/0Bz9_0VdXvv9bUUNlUTVrMF9VcVU/view?usp=sharing&resourcekey=0-16O9Fc5eaJH99-M7AHqHOg).

In [9]:
x.shape

(7380, 3)


#### TASK II
Create a function `psi(x)`, which transforms features AST (assists), REB (rebounds) and STL (steals) into 2nd order polynomial features (add each feature squared and each pair of features multiplied with every other)

**Input:** DataFrame x from above. It contains columns AST, REB, STL

**Output:** DataFrame with columns: AST, REB, STL, 1, AST^2, REB^2, STL^2, ASTSTL, REBSTL, ASTREB. The number of rows should be the same as Input.

In [7]:
def psi(x):
    poly = PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)
    poly_features = poly.fit_transform(x)
    column_names = ["AST", "REB", "STL", "1", "AST^2", "REB^2", "STL^2", "ASTSTL", "REBSTL", "ASTREB"]
    return pd.DataFrame(poly_features, columns=column_names)

In [8]:
psi_transformed = psi(x)
print(psi_transformed)

      AST   REB   STL     1   AST^2   REB^2  STL^2  ASTSTL  REBSTL  ASTREB
0     1.0  41.0  43.0  14.0  1681.0  1763.0  574.0  1849.0   602.0   196.0
1     1.0  23.0  43.0   8.0   529.0   989.0  184.0  1849.0   344.0    64.0
2     1.0  20.0  39.0   7.0   400.0   780.0  140.0  1521.0   273.0    49.0
3     1.0  19.0  47.0   6.0   361.0   893.0  114.0  2209.0   282.0    36.0
4     1.0  21.0  43.0   4.0   441.0   903.0   84.0  1849.0   172.0    16.0
...   ...   ...   ...   ...     ...     ...    ...     ...     ...     ...
7375  1.0  17.0  39.0  10.0   289.0   663.0  170.0  1521.0   390.0   100.0
7376  1.0  26.0  40.0  10.0   676.0  1040.0  260.0  1600.0   400.0   100.0
7377  1.0  23.0  52.0   8.0   529.0  1196.0  184.0  2704.0   416.0    64.0
7378  1.0  23.0  41.0  11.0   529.0   943.0  253.0  1681.0   451.0   121.0
7379  1.0  17.0  44.0   4.0   289.0   748.0   68.0  1936.0   176.0    16.0

[7380 rows x 10 columns]


#### Task III
Create a transformed data matrix **X**, where each **x** is mapped to psi(x).

HINT: We need to apply our function from Task II to matrix (DataFrame) x

#### Task IV
Create a function `p2(x,w)`, which outputs the value of the polynomial ata given row of `x` for given parameters `w`.

Inputs:
- x: DataFrame from above
- w: vector which represents beta coeficients for each column of X from the Task III.

Ouputs:
- Series of the same length as DataFrame x. Each value is a dot product of particular row in DataFrame and coeficients `w`.


HINT:
- length of w needs to be the same as number of columns of the dataframe that is returned from function `psi(x)`

Example Input:

`p2(x, [0.06, 0.05,0.03,0.01,0.02,0.02,0.04, 0.03,0.02,0.01])`

Example Output:

```
0       130.37
1        76.19
2        61.21
3        74.51
4        64.97
         ...  
7375     63.01
7376     79.59
7377     97.25
7378     78.85
7379     61.53
Length: 7380, dtype: float64

```

Our columns in the DataFrame **X** that is the output of `psi(x)` were in this order: `AST, REB, STL, 1, AST^2, REB^2, STL^2, ASTSTL, REBSTL, ASTREB`. If your columns are in different order the result can be different even for the **w**.

In [10]:
def p2(x, w):
    X_transformed = psi(x)
    result = X_transformed.dot(w)
    return pd.Series(result)

#### Task V
Create a function `Loss(x,y,w)`, which computes the squared loss of predicting **y** from **x** by `p2(x,w)` using parameters **w**. We have specified **y** as the variable PTS above. We will predict scored points (PTS) based on assists, rebounds and steals.


HINTS: 
- Output of `p2(x,w)` represents our predictions. `y_pred = p2(x,w)`
- Loss can be computed as:

```
np.mean((y_pred - y)**2)
```

#### Task VI
Code up the gradient descent. It should input **x**, target variable **y**, stepsize **alpha**, coeficients **w** and **maximum number of iterations**.

Steps:
1. transform input `x`
2. compute initial loss for given, x,y and w and append it to the empty list.
3. Inside the for loop, update each element of **w**, **w[i]**, using gradient descent.

HINT: `w[i] = w[i] - alpha * (1.0 / m) * errors_i.sum()` where `errors_i = (y_pred - y) * X.iloc[:, i]`. We are scaling the errors by multiplicating with values that are relevant for coeficients `w[i]` (column `i` of DataFrame `X`, output of `psi(x)`).

4. compute new loss using updated `w` and append to the list that we created in the step 2.
5. repeat steps 3 and 4 for max number of iterations times.

#### Task VII
Choose an arbitrary **w** and **alpha** and run gradient descent for 100 iterations. How does the loss behave? Does it converge to something?


#### Task VIII
Can you find which **alpha** from `[1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]` has the smallest loss after 100 iterations for a given **w**?